In [1]:
import pandas as pd
import glob
import numpy as np
import pathlib as pl
import os
import datetime

### Check non-PKS datasets one by one

In [2]:
PATH_TO_PROCESSED_DATA = pl.Path('../../../datasets/processed/monthly')


all_monthly_files = glob.glob("../../../datasets/raw/monthly/*.csv")
all_monthly_files

['../../../datasets/raw/monthly\\CSI.csv',
 '../../../datasets/raw/monthly\\indexofagreedearnings.csv',
 '../../../datasets/raw/monthly\\index_of_import_prices.csv',
 '../../../datasets/raw/monthly\\labour-costs.csv',
 '../../../datasets/raw/monthly\\livebirths.csv',
 '../../../datasets/raw/monthly\\marriages.csv',
 '../../../datasets/raw/monthly\\migration.csv',
 '../../../datasets/raw/monthly\\retail-trade.csv',
 '../../../datasets/raw/monthly\\unemployment-rate.csv',
 '../../../datasets/raw/monthly\\vacancies-unfilled.csv']

##### Helper functions

In [3]:
def purge_est_cols(df):
    for col in df.columns:
        if col.endswith("est"):
            df = df.drop([col],axis=1)

    return df


def mark_dupe_cols(df):
    d = {}
    new_cols = []
    for col in df.columns:
        if col in d.keys():
            new_cols.append(col + "_dupe")
        else:
            d[col] = True   
            new_cols.append(col)  
    df.columns = new_cols
    return df

def purge_dupe(df):
    for col in df.columns:
        if col.endswith("_dupe"):
            df = df.drop([col],axis=1)

    return df

def replace_unnamed_with_est(df):
    for (i, col) in enumerate(df.columns):
        if (col.startswith("Unnamed")) and i != 0:
            df.rename(columns={col: df.columns[i-1] + "_est"}, inplace=True)

    return df


def ffill_header(df, header_lvl):
    new_cols = [[0] ]* header_lvl
    for i in range(header_lvl):
        row = df.columns.get_level_values(i)
        new_row = [""] * len(row)
        good_val_met = False
        fill_word = ""
        for j in range(len(row)):
            name = row[j]

            if name.startswith("Unnamed"):
                if good_val_met:
                    new_row[j] = fill_word
                else:
                    new_row[j] = "Year" + str(j) + str(i)
            else:
                good_val_met = True
                fill_word = name
                new_row[j] = name
        new_cols[i] = new_row
    df.columns = new_cols
    return df

def set_type_to_float_from(df, col_to_start):
    for (i,col) in enumerate(df.columns):
        if i>=col_to_start:
            df[col] = df.loc[:,col].astype(float)

    return df

#first column is int (Year), rest is float
def normal_type_setup(df):
    df[df.columns[0]] = df.loc[:,df.columns[0]].astype(int)
    df = set_type_to_float_from(df,1)
    return df

def allint_type_setup(df):
    for col in df.columns:
        df[col] = df.loc[:,col].astype(int)
    return df
def set_type_to_int_from(df, col_to_start):
    for (i,col) in enumerate(df.columns):
        if i>=col_to_start:
            df[col] = df.loc[:,col].astype(int)

    return df

def prepend_col_name(df, name):
    for (i, col) in enumerate(df.columns):
        if i != 0:
            df.rename(columns={col: name + "_" +df.columns[i]}, inplace=True)


    return df

def new_column_from_cols_and_specif(col, specifier):
    col = ''.join(list(map(lambda x: str(x).strip("\n").strip("<NA>"), col)))
    clean_spec = str(specifier).strip('\n')
    new_col = f"{clean_spec}: {col}"
    new_col = new_col.replace("nan", "").replace("\\n", "")
    return new_col


def create_new_cols(specifier_column, other_columns):
    return [new_column_from_cols_and_specif(col, specifier)
                for specifier in specifier_column
                for col in other_columns
            ]

def create_transformed_df(df, specifier_column, other_columns, year):
    new_columns = create_new_cols(specifier_column, other_columns)

    # Flatten the values of all columns except the specifier column
    new_values = df.loc[:, other_columns].to_numpy().flatten()
    idx = np.array([np.issubdtype(type(x), np.floating) for x in new_values])

    filtered_values = new_values[idx]

    df_transformed = pd.DataFrame([new_values], columns=new_columns)
    
    if isinstance(year, datetime.datetime):
        df_transformed.insert(0, "Date", [year])
        df_transformed[df_transformed.columns[0]] = df_transformed.loc[:,df_transformed.columns[0]]
    else:
        df_transformed.insert(0, "Year", [year])
        df_transformed[df_transformed.columns[0]] = df_transformed.loc[:,df_transformed.columns[0]].astype(int)
    
    return df_transformed

def transform_one_df(df, specifier_key, interesting_values_basic_table, year):
    df = df[df[specifier_key].isin(interesting_values_basic_table)]

    specifier_column = df[specifier_key].to_numpy().flatten()
    other_columns = df.columns[df.columns != specifier_key]

    return create_transformed_df(df, specifier_column, other_columns, year)

def transform_all_years(df, specifier_column, other_columns):
    start_year = min(df[df.columns[0]])
    end_year = max(df[df.columns[0]])
    new_df = pd.DataFrame()
    for i in range(start_year, end_year+1):
        new_df = pd.concat([new_df, create_transformed_df(df.loc[df.Year==i,:], df.loc[df.Year==i,specifier_column], other_columns, i)],axis=0)

       
    return new_df

def mass_join(datasets, **kwargs):
    last_seen_df = None

    for tup in datasets:
        curr_df = tup[0]
        if curr_df is None:
            continue
        if last_seen_df is None:
            last_seen_df = curr_df
        else:
            if len(kwargs.keys()) > 0:
                last_seen_df = last_seen_df.merge(curr_df, on=kwargs.get("on", None), how="inner")
            else:
                last_seen_df = last_seen_df.merge(curr_df, on="Year", how="inner")
    return last_seen_df

def mass_join_till_year_x(datasets, year):
    last_seen_df = None

    for tup in datasets:
        curr_df = tup[0]
        if curr_df is None:
            continue
        if max(curr_df["Year"]) < year:
            continue
        if last_seen_df is None:
            last_seen_df = curr_df
        else:
            last_seen_df = last_seen_df.merge(curr_df, on="Year", how="inner")
    return last_seen_df

def convert_month_col(df, col):
    month_to_num = {"January" : 1, "january" : 1, "February": 2,"february": 2, "March":3, "march" :3, "April":4, "april":4, "May" : 5, "may" :5, "June":6, "june":6, "July":7, "july":7, "August":8, "august":8, "September":9,"september":9, "October":10, "october":10, "November":11,"november":11, "December":12, "december":12}
    new_col = [0]*len(col)
    new_col = df.apply(lambda row: month_to_num[row[col].strip()], axis=1)
    df.loc[:,col] = new_col

    return df

def year_month_to_datetime(df):
    df["Days"] = 1
    df["Year"] = df.loc[:,"Year"].astype(int)
    df = convert_month_col(df, "Month")
    #df["Date"] = pd.to_datetime(df[["Year", "Month", "Days"]])
    df.insert(0, "Date", pd.to_datetime(df[["Year", "Month", "Days"]]))
    df.drop(["Year", "Month", "Days"], axis=1, inplace=True)
    return df
    
def transform_all_dates(df, specifier_column, other_columns):
    start_date = df[df.columns[0]].min()
    end_date = df[df.columns[0]].max()


    new_df = pd.DataFrame()
    while (start_date < end_date):
        new_df = pd.concat([new_df, create_transformed_df(df.loc[df.Date==start_date,:], df.loc[df.Date==start_date,specifier_column], other_columns, start_date)],axis=0)
        start_date += pd.DateOffset(months=1)

       
    return new_df

        


#### Consumer Price indices

In [4]:
datasets = []
current_file = 0
df =  pd.read_csv(all_monthly_files[current_file], names=["Date", "Consumer Price Index"],skiprows=1)
df["Date"] = pd.to_datetime(df["Date"])
datasets.append((df, all_monthly_files[current_file]))
df

,Date,Consumer Price Index
0,1956-01-01,1.246106
1,1956-02-01,2.194357
2,1956-03-01,3.761755
3,1956-04-01,3.761755
4,1956-05-01,3.773585
...,...,...
822,2024-07-01,2.305722
823,2024-08-01,1.872340
824,2024-09-01,1.612903
825,2024-10-01,2.037351


#### Index of agreed earnings
Dropped hourly wages and # of working hours. Working hours were all between 37.5 - 38.5 so i felt it wasn't much info. Also dropped the "Metal and electrical industries" type of work as it had no records until 2015.

In [5]:
current_file = 1
# 
all_monthly_files[current_file]
df =  pd.read_csv(all_monthly_files[current_file],sep=";",header=[5])
df = df.loc[1:,:]
df.rename(columns={df.columns[0]:"Type code",df.columns[1]:"Type of work",(df.columns[2]):"Year",df.columns[3]:"Month"}, inplace=True)
df.drop(["Type code"],axis=1,inplace=True)

df.drop(df.tail(5).index, inplace=True)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill()
df.loc[:, "Type of work"] = df.loc[:, "Type of work"].ffill()
mask = (df.Year == 2024) & (df.Month == "December")
mask2 = df.loc[:,"Type of work"] == "Metal and electrical industries"
df = df[~mask]
df = df[~mask2]

df = year_month_to_datetime(df)
df = replace_unnamed_with_est(df)
df = purge_est_cols(df)
df.drop([df.columns[2],df.columns[3],df.columns[6],df.columns[7]],axis=1,inplace=True)

df[df.columns[2]] = df[df.columns[2]].astype(float)
df[df.columns[3]] = df[df.columns[3]].astype(float)
df = transform_all_dates(df, "Type of work", [df.columns[2], df.columns[3]])
datasets.append((df, all_monthly_files[current_file]))
df


C:\Users\KubaS\AppData\Local\Temp\ipykernel_22868\3988274002.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~mask2]


,Date,Overall economy: Index coll. agreed monthly earnings without sp.pay,Overall economy: Index coll. agreed monthly earnings with sp.pay.,Industry and service sector: Index coll. agreed monthly earnings without sp.pay,Industry and service sector: Index coll. agreed monthly earnings with sp.pay.,"Industry, trade etc.: Index coll. agreed monthly earnings without sp.pay","Industry, trade etc.: Index coll. agreed monthly earnings with sp.pay.",Business economy: Index coll. agreed monthly earnings without sp.pay,Business economy: Index coll. agreed monthly earnings with sp.pay.,"Industry, trade, transportation and storage: Index coll. agreed monthly earnings without sp.pay",...,Other personal service activities: Index coll. agreed monthly earnings without sp.pay,Other personal service activities: Index coll. agreed monthly earnings with sp.pay.,Producers of intermediate goods: Index coll. agreed monthly earnings without sp.pay,Producers of intermediate goods: Index coll. agreed monthly earnings with sp.pay.,Producers of capital goods: Index coll. agreed monthly earnings without sp.pay,Producers of capital goods: Index coll. agreed monthly earnings with sp.pay.,Producers of consumer durables: Index coll. agreed monthly earnings without sp.pay,Producers of consumer durables: Index coll. agreed monthly earnings with sp.pay.,Producers of consumer non-durables: Index coll. agreed monthly earnings without sp.pay,Producers of consumer non-durables: Index coll. agreed monthly earnings with sp.pay.
0,2010-01-01,78.0,73.4,78.0,73.4,78.3,73.0,78.7,72.7,78.5,...,79.0,75.8,78.2,71.5,78.2,70.1,79.2,71.7,78.1,76.2
0,2010-02-01,78.0,73.1,78.0,73.1,78.3,72.8,78.7,72.4,78.5,...,79.0,75.7,78.2,70.6,78.2,70.1,79.2,71.7,78.1,71.0
0,2010-03-01,78.2,73.3,78.2,73.3,78.5,72.8,78.7,72.2,78.5,...,79.0,75.7,78.2,70.5,78.2,70.1,79.2,71.0,78.2,71.1
0,2010-04-01,78.3,73.5,78.3,73.4,78.7,73.0,78.8,72.5,78.6,...,79.2,75.8,78.3,70.7,78.2,70.1,79.2,71.3,78.6,71.6
0,2010-05-01,78.3,74.2,78.3,74.2,78.7,74.2,78.8,74.1,78.6,...,79.3,76.6,78.3,73.0,78.2,74.6,79.2,74.0,78.9,72.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2024-06-01,110.0,121.1,110.0,121.1,109.4,126.5,110.5,134.9,109.6,...,111.8,119.1,109.7,136.8,109.0,157.9,108.7,145.3,113.8,121.6
0,2024-07-01,110.2,110.9,110.2,110.9,109.6,112.7,110.8,115.1,109.9,...,112.1,110.1,109.8,115.3,109.0,130.9,108.7,115.4,114.0,107.8
0,2024-08-01,110.7,107.2,110.7,107.2,110.4,107.2,111.7,108.2,111.2,...,112.7,112.9,110.1,100.2,109.0,100.6,108.8,98.9,114.8,108.5
0,2024-09-01,111.0,106.4,111.0,106.4,110.8,105.9,112.3,106.7,111.7,...,113.2,110.5,110.8,100.8,109.1,98.1,109.8,102.4,115.3,108.1


#### Index of import prices

In [6]:
current_file = 2
# 
all_monthly_files[current_file]
df =  pd.read_csv(all_monthly_files[current_file], sep=";", names=["Date", "Index of import prices"], skiprows=1)
df.loc[:,"Date"] = pd.to_datetime(df["Date"])
df["Date"] = df["Date"].apply(lambda row: row.date())
df.Date = pd.to_datetime(df.Date)
df[df.columns[1]] = df[df.columns[1]].str.replace(",",".").astype(float)
datasets.append((df, all_monthly_files[current_file]))
df

,Date,Index of import prices
0,2010-01-01,87.5
1,2010-02-01,88.2
2,2010-03-01,89.5
3,2010-04-01,91.1
4,2010-05-01,91.8
...,...,...
173,2024-06-01,113.1
174,2024-07-01,112.6
175,2024-08-01,112.2
176,2024-09-01,111.8


#### Labour costs
This is quarterly actually, it just has months to trick me apparently, skipping..

In [7]:
current_file = 3
# 
all_monthly_files[current_file]
#df =  pd.read_csv(all_monthly_files[current_file], sep=";", names=["Date", "Index of import prices"], skiprows=1)

'../../../datasets/raw/monthly\\labour-costs.csv'

#### Births 


In [8]:
current_file = 4
all_monthly_files[current_file]
df =  pd.read_csv(all_monthly_files[current_file], sep=";", skiprows=5)
df.rename(columns={df.columns[0]:"Year",df.columns[1]:"Month"}, inplace=True)
df.drop(df.tail(6).index, inplace=True)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill()

df = replace_unnamed_with_est(df)
df = purge_est_cols(df)
df = year_month_to_datetime(df)
df = prepend_col_name(df, "Births")
datasets.append((df, all_monthly_files[current_file]))
df = set_type_to_int_from(df, 1)
datasets.append((df, all_monthly_files[current_file]))
df


,Date,Births_Male,Births_Female,Births_Total
0,1950-01-01,49505,46331,95836
1,1950-02-01,47177,44298,91475
2,1950-03-01,53604,49499,103103
3,1950-04-01,49606,46030,95636
4,1950-05-01,50932,47046,97978
...,...,...,...,...
892,2024-05-01,29623,28089,57712
893,2024-06-01,28882,27242,56124
894,2024-07-01,30853,29146,59999
895,2024-08-01,29859,28257,58116


#### Marriages
I added Year and month column names to the CSV, since hte load function was acting up.

In [9]:
current_file = 5
all_monthly_files[current_file]
df =  pd.read_csv(all_monthly_files[current_file], sep=";", header=4)
df = df.loc[1:,:]
#df.rename(columns={df.columns[0]:"Year",df.columns[1]:"Month"}, inplace=True)
df.drop(df.tail(7).index, inplace=True)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill()

df = replace_unnamed_with_est(df)
df = purge_est_cols(df)

df = year_month_to_datetime(df)
datasets.append((df, all_monthly_files[current_file]))
df["Marriages"] = df["Marriages"].astype(int)
df["Marriages per 1000 inhabitants"] = df["Marriages per 1000 inhabitants"].astype(float)
datasets.append((df, all_monthly_files[current_file]))
df

,Date,Marriages,Marriages per 1000 inhabitants
1,1990-01-01,15594,2.3
2,1990-02-01,20638,3.4
3,1990-03-01,36227,5.4
4,1990-04-01,37291,5.7
5,1990-05-01,69091,10.3
...,...,...,...
413,2024-05-01,42841,6.0
414,2024-06-01,41927,6.0
415,2024-07-01,36913,5.1
416,2024-08-01,46545,6.5


#### Migration

In [10]:
current_file = 6
df =  pd.read_csv(all_monthly_files[current_file], sep=";", header=[5,6,7])
df = df.loc[1:,:]
df = ffill_header(df, 3)
df.columns = [f'{i}_{j}_{k}' for i, j,k in df.columns]
df.rename(columns={df.columns[0]:"Year", df.columns[1]:"Month"}, inplace=True)
df.drop(df.tail(6).index, inplace=True)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill()
df = mark_dupe_cols(df)
df = purge_dupe(df)
df = year_month_to_datetime(df)
datasets.append((df, all_monthly_files[current_file]))
df


,Date,Germans_Male_Arrivals from foreign countries,Germans_Male_Departures to foreign countries,Germans_Male_Migration balance,Germans_Female_Arrivals from foreign countries,Germans_Female_Departures to foreign countries,Germans_Female_Migration balance,Germans_Total_Arrivals from foreign countries,Germans_Total_Departures to foreign countries,Germans_Total_Migration balance,...,Foreigners_Total_Migration balance,Total_Male_Arrivals from foreign countries,Total_Male_Departures to foreign countries,Total_Male_Migration balance,Total_Female_Arrivals from foreign countries,Total_Female_Departures to foreign countries,Total_Female_Migration balance,Total_Total_Arrivals from foreign countries,Total_Total_Departures to foreign countries,Total_Total_Migration balance
1,2008-01-01,5327,7822,-2495,3837,6375,-2538,9164,14197,-5033,...,4862,33578,33829,-251,20580,20500,80,54158,54329,-171
2,2008-02-01,4070,6604,-2534,3107,5561,-2454,7177,12165,-4988,...,1208,29714,31503,-1789,17982,19973,-1991,47696,51476,-3780
3,2008-03-01,4399,7038,-2639,3088,5596,-2508,7487,12634,-5147,...,310,30908,34507,-3599,19680,20918,-1238,50588,55425,-4837
4,2008-04-01,4795,7066,-2271,3772,5800,-2028,8567,12866,-4299,...,12848,36369,32225,4144,23077,18672,4405,59446,50897,8549
5,2008-05-01,4503,6327,-1824,3455,5055,-1600,7958,11382,-3424,...,11593,32020,27892,4128,20325,16284,4041,52345,44176,8169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,2024-05-01,9414,12896,-3482,5385,7959,-2574,14799,20855,-6056,...,38468,80797,63532,17265,51318,36171,15147,132115,99703,32412
198,2024-06-01,9597,13305,-3708,5973,8602,-2629,15570,21907,-6337,...,33584,75885,61829,14056,49983,36792,13191,125868,98621,27247
199,2024-07-01,12547,17839,-5292,8368,12134,-3766,20915,29973,-9058,...,29212,89921,79872,10049,62084,51979,10105,152005,131851,20154
200,2024-08-01,11139,16761,-5622,7406,12211,-4805,18545,28972,-10427,...,33638,86881,75062,11819,62136,50744,11392,149017,125806,23211


#### Retail trade

In [11]:
current_file = 7
df =  pd.read_csv(all_monthly_files[current_file], sep=";")
df.loc[:,"Month"] = pd.to_datetime(df["Month"])
df["Month"] = df["Month"].apply(lambda row: row.date())
df.rename(columns={"Month":"Date"},inplace=True)
df.Date = pd.to_datetime(df.Date)
df[df.columns[1]] = df[df.columns[1]].str.replace(",",".").astype(float)
datasets.append((df, all_monthly_files[current_file]))
df

,Date,Turnover in retail trade
0,2010-01-01,78.1
1,2010-02-01,76.9
2,2010-03-01,93.2
3,2010-04-01,88.4
4,2010-05-01,87.3
...,...,...
173,2024-06-01,136.6
174,2024-07-01,142.3
175,2024-08-01,138.8
176,2024-09-01,136.4


#### Unemployment rate

In [12]:
current_file = 8
df =  pd.read_csv(all_monthly_files[current_file], sep=";")
df.loc[:,"Month"] = pd.to_datetime(df["Month"])
df["Month"] = df["Month"].apply(lambda row: row.date())
df.rename(columns={"Month":"Date"},inplace=True)
df.Date = pd.to_datetime(df.Date)
df["Unemployment rate"] = df["Unemployment rate"].str.replace(",",".").astype(float)
datasets.append((df, all_monthly_files[current_file]))
df

,Date,Unemployment rate
0,2010-01-01,8.6
1,2010-02-01,8.6
2,2010-03-01,8.5
3,2010-04-01,8.1
4,2010-05-01,7.7
...,...,...
174,2024-07-01,6.0
175,2024-08-01,6.1
176,2024-09-01,6.0
177,2024-10-01,6.0


#### Vacancies unfilled

In [13]:
current_file = 9
df =  pd.read_csv(all_monthly_files[current_file], sep=";")
df.loc[:,"Month"] = pd.to_datetime(df["Month"])
df["Month"] = df["Month"].apply(lambda row: row.date())
df.rename(columns={"Month":"Date"},inplace=True)
df.Date = pd.to_datetime(df.Date)
datasets.append((df, all_monthly_files[current_file]))
df

,Date,Vacancies unfilled
0,2010-01-01,271401
1,2010-02-01,297979
2,2010-03-01,320129
3,2010-04-01,335163
4,2010-05-01,355702
...,...,...
174,2024-07-01,703119
175,2024-08-01,698868
176,2024-09-01,696006
177,2024-10-01,688742


#### Join

In [14]:
all_monthly_pks = glob.glob("../../../datasets/raw/police_stats/monthly/*.xlsx")

In [44]:
cols_2014 = "BFGHIJKLMNOPQ"
cols_other = "BDEFGHIJKLMNO"

skiprows_2014 = 7
skiprows_other = 7

skiprows = {
    2014: 7,
    2015: 6,
    2016: 7,
    2017: 7,
    2018: 7,
    2019: 7,
    2020: 7,
    2021: 7,
    2022: 7,
    2023: 7,
}

year = "2014"
useful_col_names = ["Date"]
useful_col_names += [f"{year}-{i+1:02}-01" for i in range(len(cols_2014) - 1)]

# Assign the cleaned header and drop the original header rows

interesting_DE_to_EN_dict = {
    "Straftaten insgesamt": "Total offences",
    
    
    "Straftaten gegen das Leben": "Offences against life",
    
    
    "Mord § 211 StGB": "Murder",
    
    #"Totschlag und Tötung auf Verlangen §§ 212, 213, 216 StGB": "Manslaughter and homicide on demand §§ 212, 213, 216 StGB",
    #"Vergewaltigung und sexuelle Nötigung §§ 177 Abs. 2, 3 und 4, 178 StGB": "Rape and sexual assault §§ 177 para. 2, 3 and 4, 178 StGB",
    #"Schwerer sexueller Missbrauch von Kindern zur Herstellung und Verbreitung pornographischer Schriften § 176a Abs. 3 StGB": "Serious sexual abuse of children for the production and distribution of pornographic material Section 176a (3) StGB",
    #"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB": "Robbery, extortion and robbery of motorists §§ 249-252, 255, 316a StGB",
    #"Sonstige Raubüberfälle auf Straßen, Wegen oder Plätzen": "Other robberies on streets, roads or squares",
    "Raubüberfälle in Wohnungen": "Robberies in apartments",
    
    #"Diebstahl ohne erschwerende Umstände §§ 242, 247, 248a-c StGB und zwar:": "Theft without aggravating circumstances §§ 242, 247, 248a-c StGB",
    #"Wohnungseinbruchdiebstahl § 244 Abs. 1 Nr. 3 StGB darunter:": "Residential burglary § 244 para. 1 no. 3 StGB",
    "Diebstahl insgesamt und zwar:": "Total theft",
    
    #"Diebstahl insgesamt von Kraftwagen einschl. unbefugte Ingebrauchnahme": "Total theft of motor vehicles including unauthorized use",
    #"Diebstahl insgesamt von Fahrrädern einschl. unbefugte Ingebrauchnahme": "Total theft of bicycles including unauthorized use",
    #"Geld- und Wertzeichenfälschung, Fälschung von Zahlungskarten mit oder ohne Garantiefunktion, Schecks und Wechseln §§ 146-149, 151, 152, 152a, 152b StGB": "Counterfeiting of money, stamps, payment cards",
    "Straftaten gegen die sexuelle Selbstbestimmung insgesamt": "Criminal offenses against sexual self-determination in total",
    #"Vergewaltigung und sexuelle Nötigung §§ 177 Abs. 2, 3 und 4, 178 StGB": "Rape and sexual assault",
}

crazy_dict_DE_EN = {
    "Straftaten insgesamt": "Total offences",
    "Straftaten gegen das Leben": "Offences against life",
    "Mord § 211 StGB": "Murder",
    "Raubüberfälle in Wohnungen": "Robberies in apartments",
    "Diebstahl insgesamt und zwar:": "Total theft",
    #"Vergewaltigung und sexuelle Nötigung §§ 177 Abs. 2, 3 und 4, 178 StGB": "Rape and sexual assault",
    "Straftaten gegen die sexuelle Selbstbestimmung insgesamt": "Criminal offenses against sexual self-determination in total",
    "Straftaten gegen die sexuelle Selbstbestimmung ins": "Criminal offenses against sexual self-determination in total",
    "Straftaten gegen die sexuelle Selbstbestimmung": "Criminal offenses against sexual self-determination in total",
    "Straftaten\xa0insgesamt": "Total offences", # due to some crazy xcel stuff...
    "Straftaten\xa0gegen\xa0das\xa0Leben": "Offences against life",
    "Mord\xa0§\xa0211\xa0StGB": "Murder",
    "Raubüberfälle\xa0in\xa0Wohnungen": "Robberies in apartments",
    "Diebstahl\xa0insgesamt\xa0und\xa0zwar:": "Total theft",
    "Straftaten\xa0gegen\xa0die\xa0sexuelle\xa0Selbstbestimmung\xa0insgesamt": "Criminal offenses against sexual self-determination in total"
}

specifier_key = "Date"

df = pd.read_excel(all_monthly_pks[0], header=None, skiprows=skiprows_2014, usecols=','.join(cols_2014))
df.columns = useful_col_names

keys = list(interesting_DE_to_EN_dict.keys())
keys.extend(crazy_dict_DE_EN.keys())

df = df[df[specifier_key].str.strip().str.replace("\xa0", " ").isin(keys)]
df[specifier_key] = df[specifier_key].map(crazy_dict_DE_EN)

df = df.T
df.reset_index(inplace=True)
df.columns = df.iloc[0, :]
pks_df = df.drop(index=df.index[0]) 
pks_df.columns = ["Date", "Total offences", "Offences against life", "Murder", "Criminal offenses against sexual self-determination in total",  "Robberies in apartments", "Total theft"]

#pks_monthly_datasets = [df_2014]


pks_df

,Date,Total offences,Offences against life,Murder,Criminal offenses against sexual self-determination in total,Robberies in apartments,Total theft
1,2014-01-01,481846,211,45,3100,275,187867
2,2014-02-01,452421,187,46,2880,245,181278
3,2014-03-01,504896,211,52,3177,195,206835
4,2014-04-01,476984,204,43,2874,212,193919
5,2014-05-01,505530,227,46,3397,225,206285
6,2014-06-01,486434,218,53,3225,190,205297
7,2014-07-01,505682,224,36,3611,207,209929
8,2014-08-01,470797,217,38,2943,187,195958
9,2014-09-01,472975,185,39,2930,198,202962
10,2014-10-01,481139,185,38,2766,213,210120


In [45]:
print(f"{0}: {all_monthly_pks[0]} :: parsed cols {len(pks_df.columns)}")
for j, year in enumerate(range(2015, 2024)):
    useful_col_names = ["Date"]
    useful_col_names += [f"{year}-{i+1:02}-01" for i in range(len(cols_other) - 1)]
    # Assign the cleaned header and drop the original header rows
    df = pd.read_excel(all_monthly_pks[j+1], header=None, skiprows=skiprows[year], usecols=','.join(cols_other))
    
    df.columns = useful_col_names
    
    df = df[df[specifier_key].str.strip().str.replace("\xa0", " ").isin(keys)]
    df[specifier_key] = df[specifier_key].map(crazy_dict_DE_EN)
    
    df = df.T
    df.reset_index(inplace=True)
    df.columns = df.iloc[0, :]
    df = df.drop(index=df.index[0]) 
    df.columns = ["Date", "Total offences", "Offences against life", "Murder", "Criminal offenses against sexual self-determination in total",  "Robberies in apartments", "Total theft"]

    
    print(f"{j+1}: {all_monthly_pks[j+1]} :: parsed cols {len(df.columns)}, namely: ", list(df.columns))
    
    pks_df = pd.concat([pks_df, df])
    

pks_df



0: ../../../datasets/raw/police_stats/monthly\monthly_PKS_DE_2014.xlsx :: parsed cols 7
1: ../../../datasets/raw/police_stats/monthly\monthly_PKS_DE_2015.xlsx :: parsed cols 7, namely:  ['Date', 'Total offences', 'Offences against life', 'Murder', 'Criminal offenses against sexual self-determination in total', 'Robberies in apartments', 'Total theft']
2: ../../../datasets/raw/police_stats/monthly\monthly_PKS_DE_2016.xlsx :: parsed cols 7, namely:  ['Date', 'Total offences', 'Offences against life', 'Murder', 'Criminal offenses against sexual self-determination in total', 'Robberies in apartments', 'Total theft']
3: ../../../datasets/raw/police_stats/monthly\monthly_PKS_DE_2017.xlsx :: parsed cols 7, namely:  ['Date', 'Total offences', 'Offences against life', 'Murder', 'Criminal offenses against sexual self-determination in total', 'Robberies in apartments', 'Total theft']
4: ../../../datasets/raw/police_stats/monthly\monthly_PKS_DE_2018.xlsx :: parsed cols 7, namely:  ['Date', 'Total 

,Date,Total offences,Offences against life,Murder,Criminal offenses against sexual self-determination in total,Robberies in apartments,Total theft
1,2014-01-01,481846,211,45,3100,275,187867
2,2014-02-01,452421,187,46,2880,245,181278
3,2014-03-01,504896,211,52,3177,195,206835
4,2014-04-01,476984,204,43,2874,212,193919
5,2014-05-01,505530,227,46,3397,225,206285
...,...,...,...,...,...,...,...
8,2023-08-01,475787,209,39,7287,175,161441
9,2023-09-01,482473,192,48,7022,178,165242
10,2023-10-01,462750,165,30,5858,167,166645
11,2023-11-01,408726,154,34,4996,178,158187


In [46]:
monthly = pd.read_csv(PATH_TO_PROCESSED_DATA / "monthly.csv")

monthly

,Date,Consumer Price Index,Overall economy: Index coll. agreed monthly earnings without sp.pay,Overall economy: Index coll. agreed monthly earnings with sp.pay.,Industry and service sector: Index coll. agreed monthly earnings without sp.pay,Industry and service sector: Index coll. agreed monthly earnings with sp.pay.,"Industry, trade etc.: Index coll. agreed monthly earnings without sp.pay","Industry, trade etc.: Index coll. agreed monthly earnings with sp.pay.",Business economy: Index coll. agreed monthly earnings without sp.pay,Business economy: Index coll. agreed monthly earnings with sp.pay.,...,Total_Male_Migration balance,Total_Female_Arrivals from foreign countries,Total_Female_Departures to foreign countries,Total_Female_Migration balance,Total_Total_Arrivals from foreign countries,Total_Total_Departures to foreign countries,Total_Total_Migration balance,Turnover in retail trade,Unemployment rate,Vacancies unfilled
0,2010-01-01,0.712106,78.0,73.4,78.0,73.4,78.3,73.0,78.7,72.7,...,1412,21107,18946,2161,52970,49397,3573,78.1,8.6,271401
1,2010-02-01,0.505561,78.0,73.1,78.0,73.1,78.3,72.8,78.7,72.4,...,-144,19410,19914,-504,49355,50003,-648,76.9,8.6,297979
2,2010-03-01,1.215806,78.2,73.3,78.2,73.3,78.5,72.8,78.7,72.2,...,3707,25905,22597,3308,65896,58881,7015,93.2,8.5,320129
3,2010-04-01,1.214575,78.3,73.5,78.3,73.4,78.7,73.0,78.8,72.5,...,11185,24041,17827,6214,64208,46809,17399,88.4,8.1,335163
4,2010-05-01,1.215806,78.3,74.2,78.3,74.2,78.7,74.2,78.8,74.1,...,9562,22332,16298,6034,59063,43467,15596,87.3,7.7,355702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2024-05-01,2.403434,109.9,104.7,109.9,104.7,109.4,102.7,110.4,102.2,...,17265,51318,36171,15147,132115,99703,32412,141.1,5.8,701873
173,2024-06-01,2.226027,110.0,121.1,110.0,121.1,109.4,126.5,110.5,134.9,...,14056,49983,36792,13191,125868,98621,27247,136.6,5.8,700745
174,2024-07-01,2.305722,110.2,110.9,110.2,110.9,109.6,112.7,110.8,115.1,...,10049,62084,51979,10105,152005,131851,20154,142.3,6.0,703119
175,2024-08-01,1.872340,110.7,107.2,110.7,107.2,110.4,107.2,111.7,108.2,...,11819,62136,50744,11392,149017,125806,23211,138.8,6.1,698868


In [48]:
merged_df = pd.merge(monthly, pks_df, on='Date', how='outer') 


merged_df.to_csv(PATH_TO_PROCESSED_DATA / "monthly.csv")

merged_df 

,Date,Consumer Price Index,Overall economy: Index coll. agreed monthly earnings without sp.pay,Overall economy: Index coll. agreed monthly earnings with sp.pay.,Industry and service sector: Index coll. agreed monthly earnings without sp.pay,Industry and service sector: Index coll. agreed monthly earnings with sp.pay.,"Industry, trade etc.: Index coll. agreed monthly earnings without sp.pay","Industry, trade etc.: Index coll. agreed monthly earnings with sp.pay.",Business economy: Index coll. agreed monthly earnings without sp.pay,Business economy: Index coll. agreed monthly earnings with sp.pay.,...,Total_Total_Migration balance,Turnover in retail trade,Unemployment rate,Vacancies unfilled,Total offences,Offences against life,Murder,Criminal offenses against sexual self-determination in total,Robberies in apartments,Total theft
0,2010-01-01,0.712106,78.0,73.4,78.0,73.4,78.3,73.0,78.7,72.7,...,3573,78.1,8.6,271401,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-02-01,0.505561,78.0,73.1,78.0,73.1,78.3,72.8,78.7,72.4,...,-648,76.9,8.6,297979,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-03-01,1.215806,78.2,73.3,78.2,73.3,78.5,72.8,78.7,72.2,...,7015,93.2,8.5,320129,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-04-01,1.214575,78.3,73.5,78.3,73.4,78.7,73.0,78.8,72.5,...,17399,88.4,8.1,335163,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-05-01,1.215806,78.3,74.2,78.3,74.2,78.7,74.2,78.8,74.1,...,15596,87.3,7.7,355702,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2024-05-01,2.403434,109.9,104.7,109.9,104.7,109.4,102.7,110.4,102.2,...,32412,141.1,5.8,701873,NaN,NaN,NaN,NaN,NaN,NaN
173,2024-06-01,2.226027,110.0,121.1,110.0,121.1,109.4,126.5,110.5,134.9,...,27247,136.6,5.8,700745,NaN,NaN,NaN,NaN,NaN,NaN
174,2024-07-01,2.305722,110.2,110.9,110.2,110.9,109.6,112.7,110.8,115.1,...,20154,142.3,6.0,703119,NaN,NaN,NaN,NaN,NaN,NaN
175,2024-08-01,1.872340,110.7,107.2,110.7,107.2,110.4,107.2,111.7,108.2,...,23211,138.8,6.1,698868,NaN,NaN,NaN,NaN,NaN,NaN
